In [14]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

num_epochs = 5
num_classes = 10
batch_size = 256
learning_rate = 0.001

DATA_PATH = 'data/MNIST'
MODEL_STORE_PATH = 'data/MNIST/model'

In [15]:
trans = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

In [16]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                      batch_size=batch_size, 
                                      shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                     batch_size=batch_size,
                                     shuffle=False)

In [20]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [21]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
total_step = len(train_loader)
loss_list = []
acc_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                         (correct / total) * 100))

Epoch [1/5], Step[100/235], Loss: 0.1842, Accuracy: 95.70%
Epoch [1/5], Step[200/235], Loss: 0.2352, Accuracy: 92.19%
Epoch [2/5], Step[100/235], Loss: 0.0671, Accuracy: 98.05%
Epoch [2/5], Step[200/235], Loss: 0.0575, Accuracy: 98.44%
Epoch [3/5], Step[100/235], Loss: 0.0481, Accuracy: 97.66%
Epoch [3/5], Step[200/235], Loss: 0.0747, Accuracy: 98.05%
Epoch [4/5], Step[100/235], Loss: 0.0733, Accuracy: 99.22%
Epoch [4/5], Step[200/235], Loss: 0.0439, Accuracy: 98.05%
Epoch [5/5], Step[100/235], Loss: 0.1227, Accuracy: 97.27%
Epoch [5/5], Step[200/235], Loss: 0.0869, Accuracy: 96.88%


In [23]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {}%'.format((correct / total) * 100))
    
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.2%
